In [1]:
! pip install -U gym chainerrl

     |████████████████████████████████| 1.6MB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 19.9MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.17.1-cp36-none-any.whl size=1649757 sha256=d729035246585e198940e06ad5f6a8f63bfbde968abe478f3be54b89f82b9ac0
  Stored in directory: /home/nbuser/.cache/pip/wheels/c0/84/61/523b92d88787ae29689b3cc08cf445d8d8186d7fbe1acbf87b
Successfully built gym
  Found existing installation: pyglet 1.3.2
    Uninstalling pyglet-1.3.2:
      Successfully uninstalled pyglet-1.3.2
  Found existing installation: gym 0.15.3
    Uninstalling gym-0.15.3:
      Successfully uninstalled gym-0.15.3
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import gym
import numpy as np

In [3]:
I_am_RL_Enviroment = gym.make('CartPole-v0')

In [8]:
# Agent
import chainer
import chainerrl

In [9]:
# 因為透過微軟Azure 瀏覽器沒辦法順利產生動態的GUI
# 同學可以將這段複製到電腦上執行
# I_am_RL_Enviroment.render()
print("Observation Space : ")
print(I_am_RL_Enviroment.observation_space)
obs_size = I_am_RL_Enviroment.observation_space.shape[0]
print("Action Space : ")
print(I_am_RL_Enviroment.action_space)
n_actions = I_am_RL_Enviroment.action_space.n

Observation Space : 
Box(4,)
Action Space : 
Discrete(2)


In [10]:

# Instance of Value function Q from "chainerrl.q_functions.FCStateQFunctionWithDiscreteAction"
I_Am_Q_Function = chainerrl.q_functions.FCStateQFunctionWithDiscreteAction(obs_size, n_actions,n_hidden_layers=4, n_hidden_channels=50)

In [11]:
# Instance of training optimizer from chainer.optimizers
optimizer = chainer.optimizers.Adam(eps=1e-2)
# set the oprimizer 
optimizer.setup(I_Am_Q_Function)

In [12]:

# Set the discount factor that discounts future rewards.
gamma = 0.9

# Use epsilon-greedy for exploration
explorer = chainerrl.explorers.ConstantEpsilonGreedy(
    epsilon=0.3, random_action_func=I_am_RL_Enviroment.action_space.sample)

# DQN uses Experience Replay.
# Specify a replay buffer and its capacity.
replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10 ** 6)

# Since observations from CartPole-v0 is numpy.float64 while
# Chainer only accepts numpy.float32 by default, specify
# a converter as a feature extractor function phi.
phi = lambda x: x.astype(np.float32, copy=False)

# Now create an agent that will interact with the environment.
I_am_DQN_Agent = chainerrl.agents.DoubleDQN(
    I_Am_Q_Function, optimizer, replay_buffer, gamma, explorer,
    replay_start_size=500, update_interval=1,
    target_update_interval=100, phi=phi)

In [13]:
for i_episode in range(100): #how many episodes you want to run
    
    Observation_State = I_am_RL_Enviroment.reset() #reset() returns initial observation
    Step_Reward = 0
    Empirical_Reward = 0
    done = False
    
    for Step_T in range(100):
        # I_am_RL_Enviroment.render()
        
        print("I am step : {}".format(Step_T))
        print("============== Current Senario ==============")        
        print("Observation Space : ")
        print(I_am_RL_Enviroment.observation_space)
        print("Action Space : ")
        print(I_am_RL_Enviroment.action_space)

        
        print("============== Current Interaction  ==============")
        print("Current Observation : ")
        print(Observation_State)
        print("Current Action Select : ")
        #Action_Select = I_am_RL_Enviroment.action_space.sample()
        Action_Select = I_am_DQN_Agent.act_and_train(Observation_State , Step_Reward)
        print(Action_Select)
        print("============== Current Interaction  ==============")

        
        print("============== Enviroment Interaction Result ==============")
        Observation_State, Step_Reward, Done, Infomation = I_am_RL_Enviroment.step(Action_Select)
        
        print("New Observation State from Enviroment Interaction  : ")
        print(Observation_State)
        
        print("Reward from Enviroment Interaction  : ")
        # x 是车的水平位移, 所以 r1 是车越偏离中心, 分越少
        # theta 是棒子离垂直的角度, 角度越大, 越不垂直. 所以 r2 是棒越垂直, 分越高
        x, x_dot, theta, theta_dot = Observation_State
        reward_Position = (I_am_RL_Enviroment.x_threshold - abs(x))/I_am_RL_Enviroment.x_threshold - 0.8
        reward_Angle = (I_am_RL_Enviroment.theta_threshold_radians - abs(theta))/I_am_RL_Enviroment.theta_threshold_radians - 0.5
        Step_Reward = reward_Position + reward_Angle  # 总 reward 是 r1 和 r2 的结合, 既考虑位置, 也考虑角度, 这样 DQN 学习更有效率
        print(Step_Reward)
        Empirical_Reward = Empirical_Reward + Step_Reward

        print("Infomation from Enviroment Interaction  : ")        
        print(Infomation)
        print("============== Enviroment Interaction Result ==============")
        
        if Done :
            print(" End of Step "*10)
            break
    
    print('episode:', i_episode, 'R:', Empirical_Reward, 'statistics:', I_am_DQN_Agent.get_statistics())
    I_am_DQN_Agent.stop_episode_and_train(Observation_State, Step_Reward, done)

I am step : 0
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.04123572 -0.03572018  0.00090996 -0.01329672]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.04195012  0.15938871  0.00064402 -0.30569241]
Reward from Enviroment Interaction  : 
0.6794458098496122
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 1
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.04195012  0.15938871  0.00064402 -0.30569241]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interact

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.0343954  -0.4277136  -0.03519451  0.57585622]
Reward from Enviroment Interaction  : 
0.51762718960665
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 2
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.0343954  -0.4277136  -0.03519451  0.57585622]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.02584113 -0.62232498 -0.02367738  0.85724747]
Reward from Enviroment Interaction  : 
0.5761816880989534
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interact

============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.04660027 -1.20720944  0.13722979  1.90141067]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.07074446 -1.40352351  0.175258    2.23333244]
Reward from Enviroment Interaction  : 
-0.16627217371323433
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 9
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.07074446 -1.40352351  0.175258    2.23333244]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result =

Current Observation : 
[-0.00744577  0.18181805  0.00241589 -0.29925664]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.00380941 -0.01333826 -0.00356924 -0.00581276]
Reward from Enviroment Interaction  : 
0.6813708646882216
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 4
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.00380941 -0.01333826 -0.00356924 -0.00581276]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.00407618 -0.20840884 -0.0036855   0.28574189]
Reward from Enviroment Interaction  : 

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.01541842 -1.17267113  0.07670987  1.6073023 ]
Reward from Enviroment Interaction  : 
0.32731302384710015
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 22
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.01541842 -1.17267113  0.07670987  1.6073023 ]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.03887184 -1.36861155  0.10885591  1.9228792 ]
Reward from Enviroment Interaction  : 
0.16405470057412191
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Int

...

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.02022993 -0.15144718  0.01443195  0.26247951]
Reward from Enviroment Interaction  : 
0.6226633931858594
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 27
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.02022993 -0.15144718  0.01443195  0.26247951]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.01720098  0.04346583  0.01968154 -0.0256167 ]
Reward from Enviroment Interaction  : 
0.5988605057586541
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Inter

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.044485   -0.15595195  0.01312579  0.36185125]
Reward from Enviroment Interaction  : 
0.6187935689871029
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 47
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.044485   -0.15595195  0.01312579  0.36185125]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.04136596 0.038981   0.02036281 0.07333588]
Reward from Enviroment Interaction  : 
0.5855389197963063
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interacti

1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.07447385  0.23067508  0.01404471 -0.14376106]
Reward from Enviroment Interaction  : 
0.6019106685961154
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 61
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.07447385  0.23067508  0.01404471 -0.14376106]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.07908735 0.03535484 0.01116949 0.15331942]
Reward from Enviroment Interaction  : 
0.6137165460250276
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interacti

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.18890961  0.42529075 -0.06971806 -0.42616458]
Reward from Enviroment Interaction  : 
0.2884084622608417
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 80
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.18890961  0.42529075 -0.06971806 -0.42616458]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.19741543  0.62132737 -0.07824135 -0.73998567]
Reward from Enviroment Interaction  : 
0.2441686525737179
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Inter

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.13371459 -0.14673454  0.02118148  0.15865517]
Reward from Enviroment Interaction  : 
0.5431514551561402
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 97
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.13371459 -0.14673454  0.02118148  0.15865517]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.1307799  -0.34215324  0.02435459  0.45794436]
Reward from Enviroment Interaction  : 
0.5292237901359376
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Inter

In [14]:
for i_episode in range(10): #how many episodes you want to run
    
    Observation_State = I_am_RL_Enviroment.reset() #reset() returns initial observation
    Step_Reward = 0
    Empirical_Reward = 0
    done = False
    
    for Step_T in range(1000):
        #I_am_RL_Enviroment.render()
        
        print("I am step : {}".format(Step_T))
        print("============== Current Senario ==============")        
        print("Observation Space : ")
        print(I_am_RL_Enviroment.observation_space)
        print("Action Space : ")
        print(I_am_RL_Enviroment.action_space)

        
        print("============== Current Interaction  ==============")
        print("Current Observation : ")
        print(Observation_State)
        print("Current Action Select : ")
        #Action_Select = I_am_RL_Enviroment.action_space.sample()
        Action_Select = I_am_DQN_Agent.act(Observation_State )
        print(Action_Select)
        print("============== Current Interaction  ==============")

        
        print("============== Enviroment Interaction Result ==============")
        Observation_State, Step_Reward, Done, Infomation = I_am_RL_Enviroment.step(Action_Select)
        
        print("New Observation State from Enviroment Interaction  : ")
        print(Observation_State)
        
        print("Reward from Enviroment Interaction  : ")
        # x 是车的水平位移, 所以 r1 是车越偏离中心, 分越少
        # theta 是棒子离垂直的角度, 角度越大, 越不垂直. 所以 r2 是棒越垂直, 分越高
        x, x_dot, theta, theta_dot = Observation_State
        reward_Position = (I_am_RL_Enviroment.x_threshold - abs(x))/I_am_RL_Enviroment.x_threshold - 0.8
        reward_Angle = (I_am_RL_Enviroment.theta_threshold_radians - abs(theta))/I_am_RL_Enviroment.theta_threshold_radians - 0.5
        Step_Reward = reward_Position + reward_Angle  # 总 reward 是 r1 和 r2 的结合, 既考虑位置, 也考虑角度, 这样 DQN 学习更有效率
        print(Step_Reward)
        Empirical_Reward = Empirical_Reward + Step_Reward

        print("Infomation from Enviroment Interaction  : ")        
        print(Infomation)
        print("============== Enviroment Interaction Result ==============")
        
        if Done :
            print(" End of Step "*10)
            break
    
    print('episode:', i_episode, 'R:', Empirical_Reward, 'statistics:', I_am_DQN_Agent.get_statistics())
    I_am_DQN_Agent.stop_episode()

I am step : 0
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.04115643  0.00525683 -0.0484535   0.01439081]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.04126157 -0.18913796 -0.04816568  0.29140101]
Reward from Enviroment Interaction  : 
0.4528335036720784
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 1
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.04126157 -0.18913796 -0.04816568  0.29140101]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interact

{}
============== Enviroment Interaction Result ==============
I am step : 19
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.00671386  0.20183541  0.01408321 -0.31047936]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[-0.00267715  0.00651567  0.00787362 -0.01338848]
Reward from Enviroment Interaction  : 
0.6612907511477208
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 20
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[-0.00267715  0.00651567  0.00787362 -0.01338848]
Current Action Select : 
0
============== Curre

Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 44
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[0.00772954 0.00454197 0.00133472 0.03015079]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.00782038  0.19964476  0.00193774 -0.26211072]
Reward from Enviroment Interaction  : 
0.6874894959152444
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 45
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.00782038  0.19964476  0.00193774 -0.26211072]
Current 

============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.01769932  0.1992016   0.00704259 -0.25234377]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.02168336 0.0039798  0.00199572 0.04255218]
Reward from Enviroment Interaction  : 
0.6814364229155344
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 70
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[0.02168336 0.0039798  0.00199572 0.04255218]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==========

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.03879115  0.00343535 -0.00232737  0.05456283]
Reward from Enviroment Interaction  : 
0.6727246245367379
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 90
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.03879115  0.00343535 -0.00232737  0.05456283]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.03885986 -0.19165315 -0.00123612  0.34651054]
Reward from Enviroment Interaction  : 
0.6779063630905684
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Inter

...

0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.2733395  0.19759954 0.00821509 0.023556  ]
Reward from Enviroment Interaction  : 
0.5468843811312902
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 95
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[0.2733395  0.19759954 0.00821509 0.023556  ]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.2772915   0.39260272  0.00868621 -0.26652369]
Reward from Enviroment Interaction  : 
0.5429882855718454
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction R

============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.39831131  0.386773    0.01910626 -0.13793449]
Reward from Enviroment Interaction  : 
0.44281126562055595
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 122
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.39831131  0.386773    0.01910626 -0.13793449]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.40604677 0.19138267 0.01634757 0.1607144 ]
Reward from Enviroment Interaction  : 
0.4527599311922951
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 123
==========

Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 150
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.5812166   0.76649987  0.04315061 -0.49185098]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.59654659  0.57079667  0.03331359 -0.18588709]
Reward from Enviroment Interaction  : 
0.292378256652276
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 151
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.59654659  0.57079667  0.03331359 -0.18588709]
Cur

Current Observation : 
[0.77044658 0.17547943 0.00651996 0.51150997]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[ 0.77395617 -0.01973375  0.01675016  0.80624038]
Reward from Enviroment Interaction  : 
0.2975421418063562
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 168
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 0.77395617 -0.01973375  0.01675016  0.80624038]
Current Action Select : 
1
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[0.77356149 0.17515464 0.03287497 0.51887311]
Reward from Enviroment Interaction  : 
0.220

Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[ 1.13743462  0.74991138  0.00360035 -0.12536844]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 
[1.15243285e+00 5.54738034e-01 1.09298245e-03 1.68448176e-01]
Reward from Enviroment Interaction  : 
0.2146010386216961
Infomation from Enviroment Interaction  : 
{}
============== Enviroment Interaction Result ==============
I am step : 199
============== Current Senario ==============
Observation Space : 
Box(4,)
Action Space : 
Discrete(2)
============== Current Interaction  ==============
Current Observation : 
[1.15243285e+00 5.54738034e-01 1.09298245e-03 1.68448176e-01]
Current Action Select : 
0
============== Current Interaction  ==============
============== Enviroment Interaction Result ==============
New Observation State from Enviroment Interaction  : 